In [ ]:
import pandas as pd
import mahotas
import histomicstk as htk
import cv2
from PIL import Image, ImageOps

In [ ]:
# Reads nuclei feature map
df = pd.read_csv("TCGA14_nuclei_features.csv")
df2 = df.copy() # make copy for when we modify the dataframe

In [ ]:
# Convert cell types into intergers, more will be added later 
cell_map = {'Oligodendrocytes': 0, 'Pyramidal cell': 1, 'Psudopallisading cells': 2}

# Map cell_types as integers
df2['target'] = df['cell_type'].map(cell_map)
y = df2['target']

# Generate array of features
df_array = np.array(df2)
X = df_array[:, (0,1,2,3)] # Can modify which features to include/exclude by changing the values in ()

# split the dataset in train / test folds
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0) # Can remove random_state if reproducibility is not the main focus

In [ ]:
# Import the LogisticRegression class, create a model and fit it to training data
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(penalty='l2', max_iter=5000) # Can tune penalty to get optimized results
clf1.fit(Xtrain, ytrain)
ypred1 = clf1.predict(Xtest) # Predict the test set

In [ ]:
# Import RFC to work in
from sklearn.ensemble import RandomForestClassifier
estimators, depth = 50,20 # Can tune estimators, depth, max_features to get optimized results
clf2 = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=0)
clf2.fit(Xtrain, ytrain)
ypred2 = clf2.predict(Xtest) # Predict the test set

In [ ]:
# Import the KNeighbours classifier
from sklearn.neighbors import KNeighborsClassifier
clf3 = KNeighborsClassifier(n_neighbors=1) # Can tune neighbours to optimize results
clf3.fit(Xtrain, ytrain)
ypred3 = clf3.predict(Xtest) # Predict the test set

In [ ]:
# Import the SVM classifier
from sklearn.svm import SVC  # "Support Vector Classifier"
clf4 = SVC(kernel='linear') # Can tune type of kernel to optimize results
clf4.fit(Xtrain, ytrain)
ypred4 = clf4.predict(Xtest) # Predict the test set

In [ ]:
# Import the Kmeans clustering. This method is unsupervised, therefore it does not require the target, only features is needed
from sklearn.cluster import KMeans
est = KMeans(4)  # Can tune number of clusters to optimize results
est.fit(X) # Changed to the whole dataset (X) as we don't have any need for validation here

y_kmeans = est.predict(X) # Generate clusters here
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='rainbow'); #visualize the resutls

In [ ]:
# Display the confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(
    ytest, ypred1, cmap=plt.cm.Blues, display_labels=cell_map) # Change ypred here to visualize confusion matrices
plt.grid(False)
plt.xticks(rotation=15, ha="right")